In [1]:
''''''
import numpy as np
from matplotlib import pyplot as plt
from cnld.api import *
from cnld import fem, bem, util

%matplotlib qt

geom = define.circular_cmut_1mhz_geometry()
# print(geom)

layout = define.matrix_layout(nx=1, ny=1, pitch_x=100e-6, pitch_y=100e-6)
layout.geometries.append(geom)
layout.controldomains = define.generate_controldomainlist(layout)

grids = grid.generate_grids_from_layout(layout, refn=9)
# grids.bem.plot()



In [2]:
dbsolver = solve.DatabaseSolver('single_circular_cmut_1mhz.db', freq_interp=4, overwrite=False)
dbsolver.layout = layout
dbsolver.grids = grids
dbsolver.freqs = 0, 50e6, 500e3


dbsolver.solve()

Running:   0%|          | 0/100 [00:00<?, ?it/s]

In [10]:
from tqdm.notebook import tqdm

pulse, _ = util.gausspulse(1e6, 0.25, fs=250e6)
pulse *= 10
transmit = define.Transmit()
transmit.waveforms.append(define.Waveform(voltage=pulse))
transmit.fs = 250e6

n = 1
vmax = np.max(pulse)
e_0 = 8.85418782e-12
pmax = e_0 / 2 * vmax**2 / (geom.gap / geom.eps_r)**2 * 2
k = pmax / (1e-9)**n
lmda = 3 / 2 * k * 1

geom.contact_k = k
geom.contact_n = n
geom.contact_z0 = -geom.gap
geom.contact_lmda = lmda
layout.geometries = define.GeometryList([geom])

tsolver = solve.TimeSolver()
tsolver.layout = layout
tsolver.transmit = transmit
tsolver.dbfile = 'single_circular_cmut_1mhz.db'
tsolver.times = 0, 10e-6, 1e-9

tsolver.recalculate_fir(interp=16)
tsolver.setup()
for i in tqdm(tsolver):
    pass

0it [00:00, ?it/s]

In [11]:
plt.plot(tsolver.displacement)

In [22]:
from scipy import fftpack
import scipy as sp

def qfft(s, n=None, fs=1, axis=-1, full=False):

    ndim = s.ndim
    shape = s.shape
    nsample = shape[axis]

    if n is None:
        n = nsample

    if n > nsample:
        pw = [(0, 0)] * ndim
        pw[axis] = (0, n - nsample)
        s = np.pad(s, pw, mode='constant')
    elif n < nsample:
        s = np.take(s, range(n), axis=axis)

    ft = sp.fftpack.fft(s, axis=axis)
    freqs = sp.fftpack.fftfreq(n, 1 / fs)

    cutoff = (n + 1) // 2

    if full:
        return freqs, ft
    else:
        return freqs[:cutoff], np.take(ft, range(cutoff), axis=axis)


def qesd(s, n=None, fs=1, axis=-1, win=None):
    if win is None:
        win = np.ones(s.shape[axis])

    dims = np.ones(s.ndim, int)
    dims[axis] = -1
    win = win.reshape(dims)

    freqs, ft = qfft(s * win, n=n, fs=fs, full=False, axis=axis)
    return freqs, np.abs(ft)**2 * 2 / fs**2

fir = tsolver._fir
fir_t = tsolver._fir_t

fs = 1 / (fir_t[1] - fir_t[0])
freqs, esd = qesd(fir[0,0, :], n=4096, fs=fs)

plt.plot(freqs / 1e6, 10 * np.log10(esd / esd.max()))
plt.xlim(0, 50)

(0.0, 50.0)

In [28]:
from cnld import database

freqs, ppfr = database.read_patch_to_patch_freq_resp('single_circular_cmut_1mhz.db')

fr = ppfr[0, 1, :]
plt.plot(freqs / 1e6, 10 * np.log10(np.abs(fr) / np.abs(fr).max()), '.-')
plt.xlim(0, 50)

(0.0, 50.0)